# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')


In [3]:
display(data)
data.info()
data.shape
data.duplicated().sum()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


0

In [4]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Мы получили данные, сделали обзор и убедились, что данные не требуют обработки.

## Разбейте данные на выборки

Разобьем выборку по принципу 60/20/20.

In [5]:
r_s  = 4 
train_data, vnt_data = train_test_split(data, test_size=0.40, random_state=r_s)  
valid_data, test_data = train_test_split(vnt_data, test_size=0.50, random_state=r_s)

print('Размер тренериующей выборки', train_data.shape[0])
print('Размер валидационной выборки', valid_data.shape[0])
print('Размер тестовой выборки', test_data.shape[0])

# Я подставлял разные значения в r_s, стремясь получить максимальный показатель accuracy.
# 17       Лучшая глубина дерева 3 Лучшая точность 0.807153965785381
# 0        Лучшая глубина дерева 7 Лучшая точность 0.8133748055987559
# 4        Лучшая глубина дерева 11 Лучшая точность 0.8227060653188181

Размер тренериующей выборки 1928
Размер валидационной выборки 643
Размер тестовой выборки 643


In [6]:
train_data_features = train_data.drop(['is_ultra'], axis=1)
train_data_target = train_data['is_ultra']
valid_data_features = valid_data.drop(['is_ultra'], axis=1)
valid_data_target = valid_data['is_ultra']
test_data_features = test_data.drop(['is_ultra'], axis=1)
test_data_target = test_data['is_ultra']

Вывод по шагу 2

Данные были разбиты по принципу 60/20/20.

- Обучение пройдет на данных train_data
- Валидация модели на данных valid_data
- Лучшая модель по валидации будет применена на данных test_data

## Исследуйте модели

Обучаем модель **Дерево Решений**

In [7]:
best_depth_tree = 0
best_accuracy_tree = 0
r_s_tree = 109

for depth in range(1,101):
    model = DecisionTreeClassifier(random_state=r_s_tree, max_depth=depth)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(valid_data_features)
    accuracy = accuracy_score(valid_data_target, valid_predictions)
    #print('Глубина дерева', depth,'Точность',accuracy)
    if accuracy > best_accuracy_tree:
        best_depth_tree = depth
        best_accuracy_tree = accuracy
print()
print('Лучшая глубина дерева', best_depth_tree,'Лучшая точность', best_accuracy_tree) 

# 17   Лучшая глубина дерева 10 Лучшая точность 0.8227060653188181
# 666  Лучшая глубина дерева 10 Лучшая точность 0.8242612752721618
# 109  Лучшая глубина дерева 10 Лучшая точность 0.8273716951788491



Лучшая глубина дерева 10 Лучшая точность 0.8273716951788491


Обучаем модель **Случайный Лес**

In [8]:
best_estim_random = 0
best_accuracy_random = 0
random_state_random = 17
for estim in range(1,101):
    model = RandomForestClassifier(random_state=random_state_random, n_estimators=estim)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(valid_data_features)
    accuracy = accuracy_score(valid_data_target, valid_predictions)
    #print('Количество деревьев', estim,'Точность',accuracy)
    if accuracy > best_accuracy_random:
        best_estim_random = estim
        best_accuracy_random = accuracy

print('Лучшая количество деревьев', best_estim_random,'Лучшая точность', best_accuracy_random)  

# 1    Лучшая количество деревьев 65 Лучшая точность 0.838258164852255
# 109  Лучшая количество деревьев 36 Лучшая точность 0.8413685847589425
# 17   Лучшая количество деревьев 73 Лучшая точность 0.8429237947122862

Лучшая количество деревьев 73 Лучшая точность 0.8429237947122862


Обучаем модель **Регрессии**

In [9]:
best_max_iter = 0
best_accuracy = 0
random_state_regr = 123456

for max_iter in range(1,101):
    model = LogisticRegression(random_state=random_state_regr, max_iter=max_iter)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(valid_data_features)
    accuracy = accuracy_score(valid_data_target, valid_predictions)
    #print('Количество итерраций', max_iter,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_max_iter = max_iter
        best_accuracy = accuracy

print('Лучшая количество итерраций', best_max_iter,'Лучшая точность', best_accuracy)      

# 777  Лучшая количество итерраций 25 Лучшая точность 0.7293934681181959


Лучшая количество итерраций 25 Лучшая точность 0.7293934681181959


**Вывод по шагу 3**

В ходе исследования были получены следующие данные:

- Точность модели решающего дерева при глубине 10 равна 0.8273.
- Точность модели случайного леса при количестве деревьев 73 равна 0.8429.
- Точность модели логистической регресии равна 0.7293.
- Лучший результат показывает случайный лес с гиперпараметром max_depth=73.

## Проверьте модель на тестовой выборке

In [10]:
best_tree_model = DecisionTreeClassifier(random_state=r_s_tree, max_depth=best_depth_tree)
best_tree_model.fit(train_data_features, train_data_target)
valid_predictions = best_tree_model.predict(valid_data_features)
accuracy = accuracy_score(valid_data_target, valid_predictions)

test_predictions = best_tree_model.predict(test_data_features)
accuracy = accuracy_score(test_data_target, test_predictions)
print('Точность модели дерева решений на тестовой выборке',accuracy)

Точность модели дерева решений на тестовой выборке 0.7729393468118196


In [11]:
best_forest_model = RandomForestClassifier(random_state=random_state_random, n_estimators=best_estim_random)
best_forest_model.fit(train_data_features, train_data_target)
valid_predictions = best_forest_model.predict(valid_data_features)
accuracy = accuracy_score(valid_data_target, valid_predictions)

test_predictions = best_forest_model.predict(test_data_features)
accuracy = accuracy_score(test_data_target, test_predictions)
print('Точность модели случайного леса на тестовой выборке',accuracy)

Точность модели случайного леса на тестовой выборке 0.7884914463452566


In [12]:
best_reg_model = LogisticRegression(random_state=random_state_regr)
best_reg_model.fit(train_data_features, train_data_target)
valid_predictions = best_reg_model.predict(valid_data_features)
accuracy = accuracy_score(valid_data_target, valid_predictions)

test_predictions = best_reg_model.predict(test_data_features)
accuracy = accuracy_score(test_data_target, test_predictions)
print('Точность модели логистической регресиии на тестовой выборке',accuracy)

Точность модели логистической регресиии на тестовой выборке 0.7262830482115086


**Вывод по шагу 4**

Самый высокий показатель по Случайному Лесу.
Наш лучший результат 0,7885, что на 0,0385 выше требуемого уровня.

## (бонус) Проверьте модели на адекватность

In [13]:
test_predictions = best_forest_model.predict(test_data_features)
accuracy = accuracy_score(test_data_target, test_predictions)
precision = precision_score(test_data_target, test_predictions)
recall = recall_score(test_data_target, test_predictions)
print('Accuracy =', accuracy, 'Precision =', precision, 'Recall =', recall)

Accuracy = 0.7884914463452566 Precision = 0.7338129496402878 Recall = 0.5074626865671642


Для оценки адеватности используется F-мера - среднее гармоническое между precision и recall. Если хотя бы один из параметров близок к нулю, то и F-мера стремится к 0. Если оба стремятся к 1, то F-мера тоже стремится к 1.

In [14]:
f_score = f1_score(test_data_target, test_predictions)
print('F-мера =', f_score)

F-мера = 0.6


**Вывод по шагу 5**

Адекватность модели оценивается по нескольким параметрам. Для модели классификации это accuracy, precision и recall. Качество модели лучше всего отражают precision и recall и эти метрики складываются в F-мера как единую оценку модели.

Модель получилась среднего качества. И для таких задач она адекватна.